#Las variables a tener en cuenta: 
1. Cpu

2. Ram

3. Memory

4. Gpu

5. Weight

## Datos

1. Company- String -Laptop Manufacturer  

2. Product -String -Brand and Model  

3. TypeName -String -Type (Notebook, Ultrabook, Gaming, etc.) 

4. Inches -Numeric- Screen Size  

5. ScreenResolution -String- Screen Resolution  

6. Cpu- String -Central Processing Unit (CPU) 

7. Ram -String- Laptop RAM  

8. Memory -String- Hard Disk / SSD Memory  

9. GPU -String- Graphics Processing Units (GPU)  

10. OpSys -String- Operating System  

11. Weight -String- Laptop Weight  

12. Price_euros -Numeric- Price (Euro)  

# Bibliotecas

In [32]:
import Modelos as mod

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

## Limpieza de datos

In [33]:
#Modelo lineal 1.

#Limpiar Dataframe Train
df = pd.read_csv("train.csv", index_col = "id")
df.sort_values(by = "laptop_ID", inplace = True)
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)
df['Ram'] = df['Ram'].str.replace('GB', '').astype(float)

#Modelo Lineal:
X = df[["Weight","Ram"]]
Y = df["Price_euros"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=40)

model = LinearRegression()
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
mae = mean_absolute_error(Y_test, Y_pred)
    
print("MAE en conjunto de validación:", mae)


#Limpiar Dataframe Test:

df_test = pd.read_csv("test.csv", index_col = 0)
df_test.sort_index(inplace = True)
df_test['Weight'] = df_test['Weight'].str.replace('kg', '').astype(float)
df_test['Ram'] = df_test['Ram'].str.replace('GB', '').astype(float)
x_test = df_test[["Weight","Ram"]]


#Aplicar el modelo:
y_pred_test = model.predict(x_test)
resultados = pd.DataFrame({'id':df_test.index,
                           "Price_euros":y_pred_test})

#Imprimir el CSV:
resultados.to_csv("Resultados/Prueba_1_linear.csv", index= False)


MAE en conjunto de validación: 347.1643380815848


In [34]:
df

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
id,,,,,,,,,,,,,
427,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8.0,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69
23,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8.0,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94
1067,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8.0,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00
74,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16.0,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45
323,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8.0,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,1313,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8.0,1TB HDD,AMD Radeon R5 M430,Linux,2.30,805.99
511,1314,HP,Stream 11-Y000na,Netbook,11.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,2.0,32GB Flash Storage,Intel HD Graphics 400,Windows 10,1.17,209.00
1193,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4.0,128GB SSD,Intel HD Graphics 520,Windows 10,1.80,638.00


# Limpieza de datos y groupby

In [35]:
df = pd.read_csv("train.csv", index_col = "id")
df.sort_values(by = "laptop_ID", inplace = True)
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)
df['Ram'] = df['Ram'].str.replace('GB', '').astype(float)

medianas_type = df.groupby('Memory')['Price_euros'].median().reset_index()
medianas_type.rename(columns={'Price_euros': 'Med_Memory'}, inplace=True)

df = df.merge(medianas_type, on='Memory', how='left')

In [36]:
df = pd.read_csv("train.csv", index_col = "id")
df.sort_values(by = "laptop_ID", inplace = True)
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)
df['Ram'] = df['Ram'].str.replace('GB', '').astype(float)

medianas = df.groupby('Company')['Price_euros'].median().reset_index()
medianas.rename(columns={'Price_euros': 'Marca_mediana'}, inplace=True)

# Merge con el DataFrame original para agregar la nueva columna
df = df.merge(medianas, on='Company', how='left')

# Mostrar el DataFrame resultante
df



,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Marca_mediana
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8.0,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8.0,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94,1339.69
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8.0,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00,914.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16.0,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45,1339.69
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8.0,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,1339.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,1313,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8.0,1TB HDD,AMD Radeon R5 M430,Linux,2.30,805.99,949.00
908,1314,HP,Stream 11-Y000na,Netbook,11.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,2.0,32GB Flash Storage,Intel HD Graphics 400,Windows 10,1.17,209.00,914.00
909,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4.0,128GB SSD,Intel HD Graphics 520,Windows 10,1.80,638.00,899.00
910,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16.0,512GB SSD,Intel HD Graphics 520,Windows 10,1.30,1499.00,899.00


In [37]:

medianas_type = df.groupby('TypeName')['Price_euros'].median().reset_index()
medianas_type.rename(columns={'Price_euros': 'Mediana_Precio_Type'}, inplace=True)

# Merge con el DataFrame original para agregar la nueva columna
df = df.merge(medianas_type, on='TypeName', how='left')
df


,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Marca_mediana,Mediana_Precio_Type
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8.0,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,1339.69,1495.0
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8.0,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94,1339.69,1495.0
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8.0,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00,914.00,689.0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16.0,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45,1339.69,1495.0
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8.0,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,1339.69,1495.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,1313,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8.0,1TB HDD,AMD Radeon R5 M430,Linux,2.30,805.99,949.00,689.0
908,1314,HP,Stream 11-Y000na,Netbook,11.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,2.0,32GB Flash Storage,Intel HD Graphics 400,Windows 10,1.17,209.00,914.00,269.0
909,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4.0,128GB SSD,Intel HD Graphics 520,Windows 10,1.80,638.00,899.00,1159.0
910,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16.0,512GB SSD,Intel HD Graphics 520,Windows 10,1.30,1499.00,899.00,1159.0


In [38]:
precio_promedio_tipo = df.groupby('TypeName')['Price_euros'].mean().sort_values(ascending=False)
print(precio_promedio_tipo)

TypeName
Workstation           2494.864444
Gaming                1764.080217
Ultrabook             1530.688148
2 in 1 Convertible    1248.470471
Notebook               768.970575
Netbook                577.992857
Name: Price_euros, dtype: float64


In [39]:
precio_promedio_empresa = df.groupby('Company')['Price_euros'].mean().sort_values(ascending=False)
print(precio_promedio_empresa)

Company
Razer        3420.666667
LG           2299.000000
Google       1879.000000
MSI          1763.828824
Apple        1557.304118
Samsung      1427.333333
Microsoft    1398.950000
Huawei       1349.000000
Toshiba      1236.594595
Dell         1175.476570
Lenovo       1099.752548
Xiaomi       1044.633333
HP           1042.357884
Asus         1034.531852
Fujitsu       729.000000
Acer          644.774800
Mediacom      295.000000
Chuwi         246.945000
Vero          224.566667
Name: Price_euros, dtype: float64


In [40]:
#Modelo 2 polinomico:

# Cargar y preparar los datos
df = pd.read_csv("train.csv", index_col="id")
df.sort_values(by="laptop_ID", inplace=True)
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)
df['Ram'] = df['Ram'].str.replace('GB', '').astype(float)


X = df[["Weight", "Ram"]]
Y = df["Price_euros"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=40)

poly = PolynomialFeatures(degree=2)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model = LinearRegression()
model.fit(X_train_poly, Y_train)

Y_pred = model.predict(X_test_poly)

mae_3 = mean_absolute_error(Y_test, Y_pred)

print("MAE:", mae_3)

# Cargar el conjunto de prueba
df_test = pd.read_csv("test.csv", index_col=0)
df_test.sort_index(inplace=True)

df_test['Weight'] = df_test['Weight'].str.replace('kg', '').astype(float)
df_test['Ram'] = df_test['Ram'].str.replace('GB', '').astype(float)

# Preparar el conjunto de prueba
x_test = df_test[["Weight", "Ram"]]
X_test_poly = poly.transform(x_test)  # Corregido aquí

y_pred_test = model.predict(X_test_poly)  # Usando el conjunto de prueba transformado correctamente

# Crear el DataFrame de resultados
resultados = pd.DataFrame({'id': df_test.index,
                           "Price_euros": y_pred_test})

resultados.to_csv("Resultados/Prueba_2_polinomica.csv", index=False)


MAE: 332.7311472207639


In [50]:

#Modelo 3 polinomico:

# Cargar y preparar los datos
df = pd.read_csv("train.csv", index_col="id")
df.sort_values(by="laptop_ID", inplace=True)
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)
df['Ram'] = df['Ram'].str.replace('GB', '').astype(float)

medianas_type = df.groupby('Memory')['Price_euros'].median().reset_index()
medianas_type.rename(columns={'Price_euros': 'Med_Memory'}, inplace=True)

df = df.merge(medianas_type, on='Memory', how='left')


X = df[["Weight", "Ram", "Med_Memory"]]
Y = df["Price_euros"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=40)

poly = PolynomialFeatures(degree=2)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model = LinearRegression()
model.fit(X_train_poly, Y_train)

Y_pred = model.predict(X_test_poly)

mae_3 = mean_absolute_error(Y_test, Y_pred)

print("MAE:", mae_3)



MAE: 291.12038449596434


In [ ]:
#Insertar la nueva columna creada copiarla en e test
#que si no se generan missingvalues

In [ ]:
# Cargar el conjunto de prueba
df_test = pd.read_csv("test.csv", index_col=0)
df_test.sort_index(inplace=True)

df_test['Weight'] = df_test['Weight'].str.replace('kg', '').astype(float)
df_test['Ram'] = df_test['Ram'].str.replace('GB', '').astype(float)




# Preparar el conjunto de prueba
x_test = df_test[["Weight", "Ram", "Med_Memory"]]
X_test_poly = poly.transform(x_test)  # Corregido aquí

y_pred_test = model.predict(X_test_poly)  # Usando el conjunto de prueba transformado correctamente

# Crear el DataFrame de resultados
resultados = pd.DataFrame({'id': df_test.index,
                           "Price_euros": y_pred_test})

resultados.to_csv("Resultados/Prueba_3_polinomica.csv", index=False)